In [20]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:\\Users\\USER\\OneDrive\\Desktop\\intents-1.json').read()
intents = json.loads(data_file)

In [21]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [22]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

55 documents
20 classes ['CUT', 'Colletotrichum', 'LFT', 'ethephon', 'goodbye', 'greeting', 'options', 'rain guard', 'start tapping', 'symptom-colour', 'symptom-leaf-fall', 'symptom-leaf-retain', 'symptom-leaves', 'symptom-petiole', 'symptom-protection', 'symptom-tender-leaf', 'symptom-type', 'symptom-when', 'symptoms-number', 'thanks']
111 unique lemmatized words ["'s", '(', ')', ',', 'any', 'anyone', 'appear', 'are', 'awesome', 'be', 'black', 'bye', 'can', 'carried', 'cause', 'chatting', 'circular', 'colletotrichum', 'colour', 'controlled', 'could', 'criterion', 'cut', 'day', 'disease', 'do', 'doe', 'during', 'either', 'ethephon', 'fall', 'for', 'frequency', 'good', 'goodbye', 'guard', 'guarding', 'harm', 'harmful', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'important', 'in', 'infected', 'infection', 'is', 'it', 'later', 'leaf', 'lft', 'like', 'low', 'many', 'mature', 'mature/immature', 'me', 'measure', 'necessary', 'need', 'next', 'nice', 'noticed', 'of', 'o

In [23]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\USER\AppData\Local\Temp\ipykernel_9276\2836083368.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [24]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [25]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h6', hist)
print("model created")

Epoch 1/200
11/11 [==============================] - 0s 2ms/step - loss: 3.0265 - accuracy: 0.0545
Epoch 2/200
11/11 [==============================] - 0s 3ms/step - loss: 2.9651 - accuracy: 0.0909
Epoch 3/200
11/11 [==============================] - 0s 2ms/step - loss: 2.8481 - accuracy: 0.1455
Epoch 4/200
11/11 [==============================] - 0s 2ms/step - loss: 2.7937 - accuracy: 0.2182
Epoch 5/200
11/11 [==============================] - 0s 2ms/step - loss: 2.7015 - accuracy: 0.3091
Epoch 6/200
11/11 [==============================] - 0s 2ms/step - loss: 2.5253 - accuracy: 0.2727
Epoch 7/200
11/11 [==============================] - 0s 2ms/step - loss: 2.4109 - accuracy: 0.3455
Epoch 8/200
11/11 [==============================] - 0s 1ms/step - loss: 2.3160 - accuracy: 0.4000
Epoch 9/200
11/11 [==============================] - 0s 2ms/step - loss: 2.1526 - accuracy: 0.3818
Epoch 10/200
11/11 [==============================] - 0s 2ms/step - loss: 2.0252 - accuracy: 0.4727
Epoch 11/

Epoch 83/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2217 - accuracy: 0.9455
Epoch 84/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1533 - accuracy: 0.9636
Epoch 85/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1027 - accuracy: 0.9818
Epoch 86/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1161 - accuracy: 0.9818
Epoch 87/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2001 - accuracy: 0.9455
Epoch 88/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0707 - accuracy: 1.0000
Epoch 89/200
11/11 [==============================] - 0s 997us/step - loss: 0.1255 - accuracy: 0.9636
Epoch 90/200
11/11 [==============================] - 0s 898us/step - loss: 0.2786 - accuracy: 0.9091
Epoch 91/200
11/11 [==============================] - 0s 896us/step - loss: 0.0728 - accuracy: 0.9636
Epoch 92/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0628 - accuracy: 0

11/11 [==============================] - 0s 1ms/step - loss: 0.0840 - accuracy: 0.9636
Epoch 164/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0374 - accuracy: 1.0000
Epoch 165/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0601 - accuracy: 0.9818
Epoch 166/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0737 - accuracy: 0.9636
Epoch 167/200
11/11 [==============================] - 0s 698us/step - loss: 0.0285 - accuracy: 1.0000
Epoch 168/200
11/11 [==============================] - 0s 897us/step - loss: 0.0955 - accuracy: 0.9636
Epoch 169/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0629 - accuracy: 1.0000
Epoch 170/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0366 - accuracy: 1.0000
Epoch 171/200
11/11 [==============================] - 0s 798us/step - loss: 0.0519 - accuracy: 0.9818
Epoch 172/200
11/11 [==============================] - 0s 798us/step - loss: 0.0597 - accuracy: 0.9